In [1]:
# %tensorflow_version 2.x 
# On Colab, need this to specify tensorflow version
import numpy as np
import pandas as pd 
from tqdm.auto import tqdm
from glob import glob
import time, gc, sys
import cv2

sys.path.append("../tools/")
from helpers import (get_n, get_dummies, resize, plot_loss,
                     MultiOutputDataGenerator, plot_acc,
                    image_from_char)

In [2]:
from tensorflow import keras
import tensorflow as tf
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.models import clone_model
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
import os
DATA_FOLDER = "../../data/"
for dirname, _, filenames in os.walk(DATA_FOLDER):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../../data/test_image_data_3.parquet
../../data/train.csv
../../data/test_image_data_1.parquet
../../data/train_image_data_2.parquet
../../data/train_image_data_3.parquet
../../data/test_image_data_2.parquet
../../data/test_image_data_0.parquet
../../data/train_image_data_0.parquet
../../data/class_map.csv
../../data/test.csv
../../data/train_image_data_1.parquet
../../data/sample_submission.csv
../../data/Kalpurush_Fonts/kalpurush-2.ttf


In [4]:
train_df_ = pd.read_csv(DATA_FOLDER+'/train.csv')
test_df_ = pd.read_csv(DATA_FOLDER + '/test.csv')
class_map_df = pd.read_csv(DATA_FOLDER + '/class_map.csv')
sample_sub_df = pd.read_csv(DATA_FOLDER + '/sample_submission.csv')

In [5]:
HEIGHT = 236
WIDTH = 236

### Number of unique values

In [6]:
print(f'Number of unique grapheme roots: {train_df_["grapheme_root"].nunique()}')
print(f'Number of unique vowel diacritic: {train_df_["vowel_diacritic"].nunique()}')
print(f'Number of unique consonant diacritic: {train_df_["consonant_diacritic"].nunique()}')

Number of unique grapheme roots: 168
Number of unique vowel diacritic: 11
Number of unique consonant diacritic: 7


In [7]:
train_df_ = train_df_.drop(['grapheme'], axis=1, inplace=False)

In [8]:
train_df_[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']] = train_df_[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].astype('uint8')

In [9]:
IMG_SIZE=64
N_CHANNELS=1

## Basic Model

In [10]:
def get_model(base_filters=32, nlevels=4, ncov=4, kernel_size=(3,3), 
              activation='relu', momentum=0.15, dropout_rate=0.3,
             pool_size=2):
    keras.backend.clear_session()
    tf.random.set_seed(2040)
    inputs = Input(shape = (IMG_SIZE, IMG_SIZE, 1))
    if activation.lower() == 'leakyrelu':
        activation = keras.layers.LeakyReLU(alpha=0.05)

    model = Conv2D(filters=base_filters, kernel_size=kernel_size, padding='SAME', 
                 activation=activation, 
                 input_shape=(IMG_SIZE, IMG_SIZE, 1))(inputs)
    for _ in range(ncov-1):
        model = Conv2D(filters=base_filters, kernel_size=kernel_size, padding='SAME', 
                   activation=activation)(model)
    model = BatchNormalization(momentum=momentum)(model)
    model = MaxPool2D(pool_size=(pool_size, pool_size), )(model)
    model = Conv2D(filters=base_filters, kernel_size=(5, 5), padding='SAME',
#     model = Conv2D(filters=base_filters, kernel_size=(3, 3), padding='SAME',
                 activation=activation)(model)
    model = Dropout(rate=dropout_rate)(model)
    for i in range(nlevels-1):
        for _ in range(ncov):
            model = Conv2D(filters=2**(i+1)*base_filters, kernel_size=kernel_size, 
                         padding='SAME', activation=activation)(model)
        model = BatchNormalization(momentum=momentum)(model)
        model = MaxPool2D(pool_size=(pool_size, pool_size))(model)
        model = Conv2D(filters=2**(i+1)*base_filters, kernel_size=(5, 5), 
#         model = Conv2D(filters=2**(i+1)*base_filters, kernel_size=(3, 3), 
                       padding='SAME', activation=activation)(model)
        model = BatchNormalization(momentum=momentum)(model)
        model = Dropout(rate=dropout_rate)(model)

    model = Flatten()(model)
    model = Dense(1024, activation = activation)(model)
    model = Dropout(rate=dropout_rate)(model)
    dense = Dense(512, activation = activation)(model)

    head_root = Dense(168, activation = 'softmax', name='root')(dense)
    head_vowel = Dense(11, activation = 'softmax', name='vowel')(dense)
    head_consonant = Dense(7, activation = 'softmax', name='consonant')(dense)

    model = Model(inputs=inputs, outputs=[head_root, head_vowel, head_consonant])
#     lr = 1e-3
#     opt = keras.optimizers.RMSprop(learning_rate=lr, rho=0.9)
#     model.compile(optimizer=opt, loss='categorical_crossentropy',  metrics=['accuracy'])
    return model

In [11]:
model = get_model(base_filters=32, activation='relu', nlevels=4, ncov=4,
                  kernel_size=(3,3), pool_size=2)
lr = 1e-3
opt = keras.optimizers.RMSprop(learning_rate=lr, rho=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy',  metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 32)   9248        conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 64, 64, 32)   9248        conv2d_1[0][0]                   
______________________________________________________________________________________________

In [12]:
# model_to_load = "./saved_models/model_more_epochs.h5"
# model = keras.models.load_model(model_to_load)
# model.summary()

In [13]:
# from tensorflow.keras.utils import plot_model
# plot_model(model, to_file='model.png');

In [14]:
# weights = {'root': 0.7, 'vowel': 0.1, 'consonant':0.2}
# lr = 5e-4
# opt = keras.optimizers.RMSprop(learning_rate=lr, rho=0.9)
# model.compile(optimizer=opt, loss='categorical_crossentropy',  metrics=['accuracy'],
#               loss_weights=weights,)

In [15]:
# Learning rate will be half after 3 epochs if accuracy is not increased
lr_scheduler = []
targets = ['root', 'vowel', 'consonant']
for target in targets:
    lr_scheduler.append(ReduceLROnPlateau(monitor=f'{target}_accuracy', 
                                            patience=3,
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001))
    
# es = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
# def exponential_decay(epoch):
#    initial_lr = 0.001
#    k = 0.1
#    lr = initial_lr * np.exp(-k*epoch)
#    return lr

# lrate = keras.callbacks.LearningRateScheduler(exponential_decay)


In [16]:
batch_size = 256
# batch_size = 512
epochs = 30

In [17]:
HEIGHT = 137
WIDTH = 236

In [18]:
# model = get_model(base_filters=48, ncov=3, pool_size=3)
# model.summary()
# model = keras.models.load_model('./saved_models/model_selu_kernel-3_cov-3_basefilter-48_pool-3_3.h5')

In [19]:
import time
histories = []

In [20]:
# for i in [3]:
for i in range(2):
    train_df = pd.merge(pd.read_parquet(DATA_FOLDER + f'/train_image_data_{i}.parquet'), train_df_, on='image_id').drop(['image_id'], axis=1)

    X_train = train_df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'], axis=1)
    X_train = resize(X_train, size=IMG_SIZE)/255

    # CNN takes images in shape `(batch_size, h, w, channels)`, so reshape the images
    X_train = X_train.values.reshape(-1, IMG_SIZE, IMG_SIZE, N_CHANNELS)

    Y_train_root = pd.get_dummies(train_df['grapheme_root']).values
    Y_train_vowel = pd.get_dummies(train_df['vowel_diacritic']).values
    Y_train_consonant = pd.get_dummies(train_df['consonant_diacritic']).values

    # print(f'Training images: {X_train.shape}')
    # print(f'Training labels root: {Y_train_root.shape}')
    # print(f'Training labels vowel: {Y_train_vowel.shape}')
    # print(f'Training labels consonants: {Y_train_consonant.shape}')


    # Divide the data into training and validation set
    x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant =\
    train_test_split(X_train, Y_train_root, Y_train_vowel, Y_train_consonant, 
#                      stratify=Y_train_root, 
                     test_size=0.08, random_state=666)
    del train_df
    del X_train
    del Y_train_root, Y_train_vowel, Y_train_consonant

    # Data augmentation for creating more training data
    datagen = MultiOutputDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        # zca_whitening=True,  # apply ZCA whitening
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=8,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.15, # Randomly zoom image 
        width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
#         horizontal_flip=True,  # randomly flip images
#         brightness_range=[0.8,1.2], # darken or brightening
        vertical_flip=False)  # randomly flip images
#         vertical_flip=True)  # randomly flip images


    # This will just calculate parameters required to augment the given data. This won't perform any augmentations
    datagen.fit(x_train)

    # Fit the model
    # history = model.fit_generator(datagen.flow(x_train, {'root': y_train_root, 'vowel': 
    history = model.fit(datagen.flow(x_train, {'root': y_train_root, 'vowel':y_train_vowel, 
                                               'consonant': y_train_consonant}, 
                                     batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test, [y_test_root, y_test_vowel, y_test_consonant]), 
                              steps_per_epoch=x_train.shape[0] // batch_size, 
                              callbacks=lr_scheduler)
    #                           callbacks=[learning_rate_reduction_root, learning_rate_reduction_vowel, learning_rate_reduction_consonant])

    histories.append(history)

    # Delete to reduce memory usage
    del x_train
    del x_test
    del y_train_root
    del y_test_root
    del y_train_vowel
    del y_test_vowel
    del y_train_consonant
    del y_test_consonant
    gc.collect()
    time.sleep(120.)


  {'root': '...', 'vowel': '...', 'consonant': '...'}
    to  
  ['...', '...', '...']
Train for 180 steps, validate on 4017 samples
Epoch 1/15
180/180 [==============================] - 29s 160ms/step - loss: 8.5894 - root_loss: 4.9096 - vowel_loss: 2.2774 - consonant_loss: 1.4024 - root_accuracy: 0.0276 - vowel_accuracy: 0.2534 - consonant_accuracy: 0.6094 - val_loss: 143.6111 - val_root_loss: 73.1214 - val_vowel_loss: 54.5542 - val_consonant_loss: 15.7925 - val_root_accuracy: 0.0124 - val_vowel_accuracy: 0.1827 - val_consonant_accuracy: 0.3659
Epoch 2/15
180/180 [==============================] - 24s 133ms/step - loss: 6.9696 - root_loss: 4.5566 - vowel_loss: 1.3743 - consonant_loss: 1.0387 - root_accuracy: 0.0391 - vowel_accuracy: 0.5259 - consonant_accuracy: 0.6463 - val_loss: 5.9826 - val_root_loss: 4.2789 - val_vowel_loss: 0.8245 - val_consonant_loss: 0.8775 - val_root_accuracy: 0.0588 - val_vowel_accuracy: 0.7122 - val_consonant_accuracy: 0.6993
Epoch 3/15
180/180 [===========


  {'root': '...', 'vowel': '...', 'consonant': '...'}
    to  
  ['...', '...', '...']
Train for 180 steps, validate on 4017 samples
Epoch 1/15
180/180 [==============================] - 24s 131ms/step - loss: 2.0027 - root_loss: 1.2608 - vowel_loss: 0.4336 - consonant_loss: 0.3082 - root_accuracy: 0.6702 - vowel_accuracy: 0.8737 - consonant_accuracy: 0.9049 - val_loss: 0.9960 - val_root_loss: 0.6778 - val_vowel_loss: 0.1775 - val_consonant_loss: 0.1385 - val_root_accuracy: 0.8096 - val_vowel_accuracy: 0.9505 - val_consonant_accuracy: 0.9597
Epoch 2/15
180/180 [==============================] - 24s 131ms/step - loss: 1.7526 - root_loss: 1.0903 - vowel_loss: 0.3881 - consonant_loss: 0.2742 - root_accuracy: 0.7055 - vowel_accuracy: 0.8825 - consonant_accuracy: 0.9157 - val_loss: 1.2739 - val_root_loss: 0.7997 - val_vowel_loss: 0.2724 - val_consonant_loss: 0.1972 - val_root_accuracy: 0.7824 - val_vowel_accuracy: 0.9422 - val_consonant_accuracy: 0.9490
Epoch 3/15
180/180 [================

In [20]:
model.save("./saved_models/model_more_epochs_with_parquet3.h5")

In [22]:
# import pickle
# with open("./saved_histories/model_more_epochs", "wb") as pf:
#     pickle.dump(histories[0].history, pf)    
#     pickle.dump(histories[1].history, pf)    
#     pickle.dump(histories[2].history, pf)
#     pickle.dump(histories[3].history, pf)    

IndexError: list index out of range

In [ ]:
# def save_model(base_filters=32, nlevels=4, ncov=4, kernel_size=(3,3), 
#               activation='relu', momentum=0.15, dropout_rate=0.3):
#   filepath = os.path.join(DATA_FOLDER, "saved_model", f'''{str(base_filters)}-{str(nlevels)}-{str(ncov)}-{str(kernel_size[0])}-{activation}-{str(momentum)}-{str(dropout_rate)}.h5''')
#   print(filepath)
#   model.save(filepath)

In [ ]:
# # save_model()
# histories = [1,2,3]
# kernel = 2
# ncov = 3
# with open(f"./saved_histories/histories-{kernel}-{ncov}", "wb") as pf:
#     pickle.dump(histories, pf)

In [ ]:
# import pickle
# with open(os.path.join(DATA_FOLDER, "saved_model", "history_basic_model_leaky"), "wb") as pf:
#   pickle.dump(histories[0].history, pf)

In [ ]:
for dataset in range(4):
    plot_loss(histories[dataset], epochs, f'Training Dataset: {dataset}')
    plot_acc(histories[dataset], epochs, f'Training Dataset: {dataset}')

In [ ]:
del histories
gc.collect()

In [ ]:
preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}

In [ ]:
components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] # model predictions placeholder
row_id=[] # row_id place holder
for i in range(4):
    df_test_img = pd.read_parquet(DATA_FOLDER + f'/test_image_data_{i}.parquet') 
    df_test_img.set_index('image_id', inplace=True)

    X_test = resize(df_test_img, size=IMG_SIZE,need_progress_bar=False)/255
    X_test = X_test.values.reshape(-1, IMG_SIZE, IMG_SIZE, N_CHANNELS)
    
    preds = model.predict(X_test)

    for i, p in enumerate(preds_dict):
        preds_dict[p] = np.argmax(preds[i], axis=1)

    for k,id in enumerate(df_test_img.index.values):  
        for i,comp in enumerate(components):
            id_sample=id+'_'+comp
            row_id.append(id_sample)
            target.append(preds_dict[comp][k])
    del df_test_img
    del X_test
    gc.collect()

df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)
df_sample.to_csv('submission.csv',index=False)
df_sample.head()

In [ ]:
df_sample